## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-16-08-50-12 +0000,nyt,Trump Leaves Tense America Behind for Royal Re...,https://www.nytimes.com/2025/09/16/world/europ...
1,2025-09-16-08-49-51 +0000,cbc,Trump launches $15B defamation US lawsuit agai...,https://www.cbc.ca/news/world/trump-lawsuit-ne...
2,2025-09-16-08-41-28 +0000,bbc,State pension likely to rise by 4.7% in April,https://www.bbc.com/news/articles/c62lnzdndkeo...
3,2025-09-16-08-40-17 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/09/16/world/...
4,2025-09-16-08-38-55 +0000,bbc,What is the triple lock and how much is the st...,https://www.bbc.com/news/articles/cq6m03ld7nvo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2442/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,54
99,kirk,31
154,charlie,30
12,new,20
32,israel,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
85,2025-09-16-00-41-00 +0000,wsj,President Trump got a win when the Senate conf...,https://www.wsj.com/economy/central-banking/st...,120
40,2025-09-16-04-22-08 +0000,nypost,Old man who falsely claimed he shot Charlie Ki...,https://nypost.com/2025/09/16/us-news/old-man-...,113
36,2025-09-16-04-57-35 +0000,bbc,Trump says he will sue New York Times for $15bn,https://www.bbc.com/news/articles/cevz419pk8vo...,111
30,2025-09-16-05-27-10 +0000,wapo,Trump files $15 billion defamation and libel l...,https://www.washingtonpost.com/business/2025/0...,109
32,2025-09-16-05-13-50 +0000,nypost,Trump to file $15 billion defamation and libel...,https://nypost.com/2025/09/16/us-news/trump-to...,109


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
85,120,2025-09-16-00-41-00 +0000,wsj,President Trump got a win when the Senate conf...,https://www.wsj.com/economy/central-banking/st...
40,113,2025-09-16-04-22-08 +0000,nypost,Old man who falsely claimed he shot Charlie Ki...,https://nypost.com/2025/09/16/us-news/old-man-...
14,68,2025-09-16-07-22-00 +0000,wsj,Israel Launches New Ground Offensive Seeking t...,https://www.wsj.com/world/middle-east/israel-l...
74,54,2025-09-16-01-14-00 +0000,wsj,A federal appeals court on Monday night reject...,https://www.wsj.com/economy/central-banking/ap...
126,45,2025-09-15-22-18-14 +0000,nypost,"US, China near TikTok deal to finally resolve ...",https://nypost.com/2025/09/15/business/us-chin...
158,40,2025-09-15-20-20-00 +0000,wsj,Tesla Shares Rise on Elon Musk’s $1 Billion St...,https://www.wsj.com/tech/tesla-stock-elon-musk...
226,37,2025-09-15-16-57-00 +0000,nypost,"On ‘Charlie Kirk Show’ hosted by JD Vance, Ste...",https://nypost.com/2025/09/15/us-news/jd-vance...
182,35,2025-09-15-18-54-19 +0000,latimes,Suspect sent text saying he would 'take out' C...,https://www.latimes.com/california/story/2025-...
242,33,2025-09-15-15-15-00 +0000,wsj,Canadian factories notched the biggest rise in...,https://www.wsj.com/articles/canada-factory-an...
72,32,2025-09-16-01-20-34 +0000,nypost,Evergreen High School student writes moving vi...,https://nypost.com/2025/09/15/us-news/evergree...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
